<a href="https://colab.research.google.com/github/jrchac/Thesis-Project-Feele/blob/main/G2020_Cleaning_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Garage2020: Project

**Changes from last cleaning code**:

- removed the outlier in daily_steps from the df (changed in the cleaning function)
- changed column 'daily_step' into 'daily_steps'
- added some description and background for the code


## Cleaning the Data

In [ ]:
#pip install tslearn

In [ ]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import matplotlib.pyplot as plt

**Background:**

While analyzing user data for clustering and creating user profiles, we identified that discrepancies exist within the gender and birthday fields. Specifically, we observed that both gender and birthday values change simultaneously within certain user groups, indicating that the data for a single user_id might actually belong to multiple distinct users. Addressing these discrepancies is crucial for accurate clustering and profile creation.


**Objective:**

To ensure the integrity of our user clustering process, we need to handle these discrepancies by distinguishing and appropriately grouping data that likely belongs to different users. This will involve:

- Identifying user records with inconsistent gender and birthday values.
- Creating new user IDs for these distinct groups within the same user_id.
- Deleting outliers.

**Approach:**

Situation 1: If user_id, gender, and birthday values are consistent (i.e., no changes in gender and birthday), we treat this data as belonging to a single user.

Situation 2: If user_id is the same but gender and birthday values change consistently together, we infer the presence of multiple users. In this case, we group the data by combinations of user_id, gender, and birthday, and assign new user IDs to distinguish these groups.

In [ ]:
def check_gender_birthday_discrepancy(data):
    # Group the data by user_id
    users = data.user_id.unique()
    # Initialize a list to store user IDs with discrepancies
    users_with_discrepancies = []

    # Iterate over each group
    for user_id in users:
        temp = data.loc[data.user_id == user_id]
        # Check if there are multiple unique values for gender or birthday
        if temp.gender.nunique() > 1 or temp.birthday.nunique() > 1:
            users_with_discrepancies.append(user_id)
    return users_with_discrepancies

In [ ]:
def create_new_users(data, users):
    new_user_id = 10_000
    clean_users = data.loc[~data.user_id.isin(users)]
    disc_users = data.loc[data.user_id.isin(users)]
    total = [clean_users]

    for user in users:
        disc_user = disc_users.loc[disc_users.user_id == user]
        tmp = []
        for i, (_, g) in enumerate(disc_user.groupby(["gender","birthday"])):
            if i:
                g["user_id"] = new_user_id
                new_user_id = new_user_id + 1
            tmp.append(g)
        total.append(pd.concat(tmp))
    return pd.concat(total).reset_index(drop=True)

In [ ]:
import datetime
def cleaning():
    df = pd.read_excel("Garage2020_dataset.xlsx")

    df.loc[38043, "gender"] = "he"
    df.loc[38043, "birthday"] = 2000.0
    df.loc[54391, "gender"] = "she"
    df.loc[54391, "birthday"] = 1969.0

    discrepancy_users = check_gender_birthday_discrepancy(df)
    new_df = create_new_users(df, discrepancy_users)

    dm = new_df.loc[~(new_df.question_1.isnull() | new_df.question_2.isnull())]
    dm = dm.loc[dm.question_4 != 'Weet ik niet']
    dm = dm.drop(columns=["os"])

    cols = ["X00.01","X01.02","X02.03","X03.04","X04.05","X05.06","X06.07","X07.08","X08.09","X09.10","X10.11","X11.12","X12.13","X13.14","X14.15","X15.16","X16.17","X17.18","X18.19","X19.20","X20.21","X21.22","X22.23","X23.00"]
    dm[cols] = dm[cols].fillna(-1)
    dm[cols] = dm[cols].astype(int)

    # Find the users who are submitting daily steps as hourly steps. Ensure that users who are not
    # submitting steps to be out of this filtering process.
    dt = dm.loc[dm.apply(lambda x: np.all(x[cols].values == x[cols].values[0]) and x[cols].values[0] != -1, axis=1)]
    # assign -1 to register those users as 'not submitting'
    dm.loc[dt.index.values, cols] = -1

    # add response_age
    dm["birth"] = dm.birthday.apply(lambda x: datetime.datetime(year=int(x), month=1,day=1))
    dm["response_age"] = ((dm.DateTime - dm.birth).dt.days//365.25).astype(int)
    dm = dm.drop(columns=["birth"])

    step_cols = [col for col in dm.columns if col.startswith("X")]
    dm["daily_steps"] = dm[step_cols].sum(axis=1).astype(int)
    dm.loc[dm.daily_steps < 0, "daily_steps"] = -1
    dm = dm.drop(columns=step_cols) #  + ["Date", "Time", "DateTime"]
    dm.reset_index(inplace=True, drop=True)

    # dropping the outlier in daily steps
    dm = dm.drop(index= (dm[dm['daily_steps'] == dm['daily_steps'].max()]).index )
    return dm


In [ ]:
data = cleaning() # takes 2 min
data.head()

,user_id,gender,birthday,Date,Time,DateTime,question_1,question_2,question_3,question_4,question_5,response_age,daily_steps
0,2527,she,1995.0,2022-04-12,14:40:20,2022-04-12 14:40:20,Niet oke,Ontevreden,Ziek,Iets anders,Klasgenoten,27,2485
1,2527,she,1995.0,2022-04-13,14:56:58,2022-04-13 14:56:58,Oke,Verrast,NaN,NaN,NaN,27,6237
2,2527,she,1995.0,2022-04-13,14:36:44,2022-04-13 14:36:44,Oke,Verrast,Oke nacht,Werken,Klasgenoten,27,6237
3,2527,she,1995.0,2022-04-14,08:00:41,2022-04-14 08:00:41,Goed,Vreugdevol,NaN,NaN,NaN,27,5379
4,2527,she,1995.0,2022-04-20,12:06:25,2022-04-20 12:06:25,Niet oke,Nerveus,Stress,Werken,Alleen,27,6025
